# Import necessary libraries

In [ ]:
# Standard python libraries
import logging
import os
import time
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.tasks import Task
from lightautoml.utils.profiler import Profiler

from lightautoml.addons.uplift.base import AutoUplift
from lightautoml.addons.uplift import meta_learners
from lightautoml.addons.uplift.metrics import (_available_uplift_modes,
                                               calculate_graphic_uplift_curve,
                                               calculate_min_max_uplift_auc,
                                               calculate_uplift_auc,
                                               perfect_uplift_curve)
from lightautoml.addons.uplift.utils import create_linear_automl
from lightautoml.report.report_deco import ReportDecoUplift


%matplotlib inline

# Parameters

## Setting

In [ ]:
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 300 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name
TREATMENT_NAME = 'TREATMENT' # Treatment column name

## Fix torch number of threads and numpy seed

In [ ]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Example data load

In [ ]:
%%time

data = pd.read_csv('./example_data/test_data_files/sampled_app_train.csv')
data.head()

# (Optional) Some user feature preparation

In [ ]:
%%time 

data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                    ).astype(str)

data['report_dt'] = np.datetime64('2018-01-01')

data['constant'] = 1
data['allnan'] = np.nan

data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

# Create treatment column with synthetic values (only for example)

In [ ]:
%%time 

data[TREATMENT_NAME] = np.nan
data.loc[data[TARGET_NAME] == 0, TREATMENT_NAME] = \
    np.random.randint(2, size=data.loc[data[TARGET_NAME] == 0].shape[0])
data.loc[data[TARGET_NAME] == 1, TREATMENT_NAME] = \
    np.random.choice([0, 1], data[data[TARGET_NAME] == 1].shape[0], p=[0.3, 0.7])

# Data splitting for train-test

In [ ]:
%%time

train, test = train_test_split(data, test_size=2000, random_state=42, shuffle=True)

test_target, test_treatment = test[TARGET_NAME].values.ravel(), test[TREATMENT_NAME].values.ravel()

# Setup columns roles

In [ ]:
%%time

roles = {
    'target': TARGET_NAME,
    'treatment': TREATMENT_NAME,
    DatetimeRole(base_date=True, seasonality=(), base_feats=False): 'report_dt'
}

# Uplift modeling

## AutoUplift (automaticaly choose best uplift method)

### Fit autouplift

In [ ]:
%%time

# Set uplift candidate for choosing best of them
# !!!ATTENTION!!!
#    This is a demonstration of the possibilities, 
#    You may use default set of candidates 
#
# Format: ('String_Name', 'MetaLearnerClass', 'Parameter for MetaLearnerClass')

task = Task('binary')

uplift_candidates = [
    (
        'TLearner__Default', 
        meta_learners.TLearner, 
        {'base_task': task}
    ),  
    (
        'XLearner__Custom',
        meta_learners.XLearner,
        {
            'outcome_learners': [
                TabularAutoML(task=task, timeout=10), # [sec] , Only speed up example, don't change it!
                create_linear_automl(base_task=Task('binary'))
            ],
            'effect_learners': [TabularAutoML(task=Task('reg'), timeout=5)],
            'propensity_learner': create_linear_automl(base_task=Task('binary')),
        }    
    )
]

autouplift = AutoUplift(Task('binary'),
                        uplift_candidates=uplift_candidates, 
                        add_dd_candidates=True,
                        metric='adj_qini', 
                        normed_metric=True, 
                        test_size=0.2)

# !!!ATTENTION!!! 
# Only speed up example, don't change it!

autouplift.__TABULAR_DEFAULT_TIME__ = 10
autouplift.__THRESHOLD_DISBALANCE_TREATMENT__ = 0.0

autouplift.fit(train, roles)

### Show rating of uplift methods (meta-learners)

In [ ]:
%%time

rating_table = autouplift.get_metalearners_ranting()
rating_table

### Predict to test data and check metrics

In [ ]:
%%time

uplift_pred, treatment_pred, control_pred = autouplift.predict(test)
uplift_pred = uplift_pred.ravel()

roc_auc_treatment = roc_auc_score(test_target[test_treatment == 1], treatment_pred[test_treatment == 1])
roc_auc_control = roc_auc_score(test_target[test_treatment == 0], control_pred[test_treatment == 0])

uplift_auc_algo = calculate_uplift_auc(test_target, uplift_pred, test_treatment, normed=False)
uplift_auc_algo_normed = calculate_uplift_auc(test_target, uplift_pred, test_treatment, normed=True)
auc_base, auc_perfect = calculate_min_max_uplift_auc(test_target, test_treatment)

logging.info('--- Check scores ---')
logging.info('OOF scores "ROC_AUC":')
logging.info('\tTreatment = %f', roc_auc_treatment)
logging.info('\tControl   = %f', roc_auc_control)
logging.info('Uplift score of test group (default="adj_qini"):')
logging.info('\tBaseline      = %f', auc_base)
logging.info('\tAlgo (Normed) = %f (%f)', uplift_auc_algo, uplift_auc_algo_normed)
logging.info('\tPerfect       = %f', auc_perfect)

###  Get best metalearner with report functionaly (should refit on train data for generating report) 

In [ ]:
%%time 

best_metalearner_repo = autouplift.create_best_meta_learner(need_report=True)
best_metalearner_repo.fit(train, roles)
best_metalearner_repo.predict(test)

# Path to report: PATH_TO_CURRENT_NOTEBOOK/lama_report/lama_interactive_report.html

## MetaLearner

### TLearner

#### Fit on train data

In [ ]:
%%time

# Default setting
tlearner = meta_learners.TLearner(base_task=Task('binary'))
tlearner.fit(train, roles)

#### Predict to test data and check metrics

In [ ]:
%%time

uplift_pred, treatment_pred, control_pred = tlearner.predict(test)
uplift_pred = uplift_pred.ravel()

roc_auc_treatment = roc_auc_score(test_target[test_treatment == 1], treatment_pred[test_treatment == 1])
roc_auc_control = roc_auc_score(test_target[test_treatment == 0], control_pred[test_treatment == 0])

uplift_auc_algo = calculate_uplift_auc(test_target, uplift_pred, test_treatment, normed=False)
uplift_auc_algo_normed = calculate_uplift_auc(test_target, uplift_pred, test_treatment, normed=True)
auc_base, auc_perfect = calculate_min_max_uplift_auc(test_target, test_treatment)

logging.info('--- Check scores ---')
logging.info('OOF scores "ROC_AUC":')
logging.info('\tTreatment = %f', roc_auc_treatment)
logging.info('\tControl   = %f', roc_auc_control)
logging.info('Uplift score of test group (default="adj_qini"):')
logging.info('\tBaseline      = %f', auc_base)
logging.info('\tAlgo (Normed) = %f (%f)', uplift_auc_algo, uplift_auc_algo_normed)
logging.info('\tPerfect       = %f', auc_perfect)

### XLearner

#### Fit on train data

In [ ]:
%%time

# Custom base algorithm
xlearner = meta_learners.XLearner(
    propensity_learner=TabularAutoML(task=Task('binary'), timeout=10),
    outcome_learners=[
        TabularAutoML(task=Task('binary'), timeout=10),
        TabularAutoML(task=Task('binary'), timeout=10)
    ],
    effect_learners=[
        TabularAutoML(task=Task('reg'), timeout=10),
        TabularAutoML(task=Task('reg'), timeout=10)
    ]
)
xlearner.fit(train, roles)

#### Predict to test data and check metrics

In [ ]:
%%time

uplift_pred, treatment_pred, control_pred = xlearner.predict(test)
uplift_pred = uplift_pred.ravel()

roc_auc_treatment = roc_auc_score(test_target[test_treatment == 1], treatment_pred[test_treatment == 1])
roc_auc_control = roc_auc_score(test_target[test_treatment == 0], control_pred[test_treatment == 0])

uplift_auc_algo = calculate_uplift_auc(test_target, uplift_pred, test_treatment, normed=False)
uplift_auc_algo_normed = calculate_uplift_auc(test_target, uplift_pred, test_treatment, normed=True)
auc_base, auc_perfect = calculate_min_max_uplift_auc(test_target, test_treatment)

logging.info('--- Check scores ---')
logging.info('OOF scores "ROC_AUC":')
logging.info('\tTreatment = %f', roc_auc_treatment)
logging.info('\tControl   = %f', roc_auc_control)
logging.info('Uplift score of test group (default="adj_qini"):')
logging.info('\tBaseline      = %f', auc_base)
logging.info('\tAlgo (Normed) = %f (%f)', uplift_auc_algo, uplift_auc_algo_normed)
logging.info('\tPerfect       = %f', auc_perfect)

# Uplift metrics and graphics (using xlearner predictions)

In [ ]:
%%time 

UPLIFT_METRIC = 'adj_qini'

logging.info("All available uplift metrics: %s", _available_uplift_modes)

## Algorithm uplift curve 

In [ ]:
%%time

# Algorithm curve
xs_xlearner, ys_xlearner = calculate_graphic_uplift_curve(
    test_target, uplift_pred, test_treatment, mode=UPLIFT_METRIC
)

## Baseline, perfect curve

In [ ]:
# Baseline curve
xs_base, ys_base = [0, 1], [0, ys_xlearner[-1]]

# Perfect curver
perfect_uplift = perfect_uplift_curve(test_target, test_treatment)
xs_perfect, ys_perfect = calculate_graphic_uplift_curve(
    test_target, perfect_uplift, test_treatment, mode=UPLIFT_METRIC)

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(xs_base, ys_base, 'black')
plt.plot(xs_xlearner, ys_xlearner, 'red')
plt.plot(xs_perfect, ys_perfect, 'green')

plt.fill_between(xs_xlearner, ys_xlearner, alpha=0.5, color='orange')

plt.xlabel('Cumulative percentage of people in T/C groups')
plt.ylabel('Uplift metric (%s)'.format(UPLIFT_METRIC))
plt.grid()
plt.legend(['Baseline', 'XLearner', 'Perfect']);

# Report

In [ ]:
%%time

RDU = ReportDecoUplift()
tlearner_deco = RDU(meta_learners.TLearner(base_task=Task('binary')))
tlearner_deco.fit(train, roles)
tlearner_deco.predict(test);

# Path to report: PATH_TO_CURRENT_NOTEBOOK/lama_report/lama_interactive_report.html